In [15]:
%pip install self-supervised -Uq

Note: you may need to restart the kernel to use updated packages.


In [16]:
from fastai.vision.all import *

from self_supervised.augmentations import *
from self_supervised.layers import *
from self_supervised.vision.swav import *

from sklearn.model_selection import StratifiedKFold
import torchvision.models as models

In [17]:
df = pd.read_csv('D:\\PhD\\files_and_targets.csv')
df

,id,image,target
0,0,anthrophony_1.png,anthrophony
1,1,anthrophony_10.png,anthrophony
2,2,anthrophony_100.png,anthrophony
3,3,anthrophony_101.png,anthrophony
4,4,anthrophony_102.png,anthrophony
...,...,...,...
6103,6103,other_995.png,other
6104,6104,other_996.png,other
6105,6105,other_997.png,other
6106,6106,other_998.png,other


Using the StratifiedKFold technique to split data into train & validation sets.

In [18]:
sk_fold = StratifiedKFold(5)
df['is_valid'] = False
for i, (trn_idx, val_idx) in enumerate(sk_fold.split(df, df.target)):
    df.loc[val_idx, 'is_valid'] = True
    break
    
df.groupby('is_valid').target.value_counts()

is_valid  target     
False     biophony       1788
          geophony       1451
          other          1222
          anthrophony     425
True      biophony        447
          geophony        363
          other           305
          anthrophony     107
Name: target, dtype: int64

In [19]:
def get_x(x): return 'D:\\train_images\\' + x['image']

def get_dls(size, bs, workers=None):
    path = Path('D:\\train_images\\')
    
    db = DataBlock(blocks = (ImageBlock(), CategoryBlock()),
              get_x = get_x, get_y=ColReader('target'),
              splitter=ColSplitter(),
              item_tfms=RandomResizedCrop(size, min_scale=1.))
    dls = db.dataloaders(df, bs=bs, num_workers=workers)
    return dls

In [20]:
bs, resize, size = 24, 256, 224
dls = get_dls(resize, bs)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck
Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `number_workers` is changed to 0 to avoid getting stuck


In [21]:
## Model
encoder = create_fastai_encoder(resnet50(), pretrained=True, n_in=3)
model = create_swav_model(encoder)
model.to("cuda")

## SwAV callback
K = bs*2**4
aug_pipelines = get_swav_aug_pipelines(num_crops=[2, 6],
                                       crop_sizes=[size,int(3/4*size)], 
                                       min_scales=[0.25, 0.20],
                                       max_scales=[1.00, 0.35],
                                       rotate=True, rotate_deg=10, jitter=True, bw=True, blur=False)
cbs=[SWAV(aug_pipelines, crop_assgn_ids=[0,1], K=K, queue_start_pct=0.5, temp=0.1)]

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
learn = Learner(dls, model, cbs=cbs)

In [ ]:
# b = dls.one_batch()
# learn._split(b)
# learn('before_batch')
# learn.swav.show(n=5)

In [ ]:
lr, wd = 1e-2, 1e-2
epochs = 100 # try using 40 or 50
learn.unfreeze()
learn.fit_flat_cos(epochs, lr, wd=wd, pct_start=0.5)

In [ ]:
save_name = f'swav_iwang_sz{size}_epc{epochs}'
learn.save(save_name)
torch.save(learn.model.encoder.state_dict(), learn.path/learn.model_dir/f'{save_name}_encoder.pth')
learn.recorder.plot_loss()